# Exploración y preprocesado de series temporales

## Importar librerías

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.graphics.tsaplots as sgt
import statsmodels.tsa.stattools as sts
from statsmodels.tsa.seasonal import seasonal_decompose

## Importar datos

El archivo Index2018.csv que vamos a importar incorpora series temporales de distintos índices bursátiles como el S&P500, por ejemplo.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
raw_csv_data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/MBIT/2025-10-MIA_Abr25/datasets/Index2018.csv")

Una vez importado, lo primero que vamos a hacer es copiarlo a una nueva variable por si en algún momento necesitamos utilizar el archivo sin transformar.

In [ ]:
df_comp = raw_csv_data.copy()

## Preprocesado

### Explorar los datos

Una vez importados los datos, vamos a exportarlos. Para ver qué contiene el dataset, vamos a usar la función head(), que muestra por defecto las 5 primeras filas del dataset.

In [ ]:
df_comp.head()

Si queremos ver todos los datos, entonces simplemente nombramos al dataset.

In [ ]:
df_comp

El siguiente paso es hacer un pequeño análisis exploratorio rápido de cómo se distribuyen los datos de cada una de las variables, para ver si hay algún dato atípico y conocer los estadísticos básicos del conjunto de datos (media, desviación estandar, percentiles, mínimos, máximos)

In [ ]:
df_comp.dtypes

In [ ]:
df_comp.describe()

Después vamos a comprobar si el conjunto de datos tiene datos faltantes. Esto es importante porque cuando veamos distintos modelos, se van a utilizar datos del pasado para el cálculo de los valores futuros y, si tenemos algún dato faltante, tendremos que trabajar en ellos previamente.

In [ ]:
df_comp.isna()

Para ver una agrupación de cuántos valores faltantes tenemos en cada una de las variables, utilizamos sum()

In [ ]:
df_comp.isna().sum()

Como ves, en estos casos no existen valores faltantes.

Podemos también contar los valores faltantes en una única variable, no es necesario hacerla sobre todas ellas

In [ ]:
df_comp['spx'].isna().sum()

In [ ]:
df_comp['spx'].isna().sum()

### Plot de los datos

Para hacer la visualización de los datos, lo primero que vamos a hacer es importar la librería matplotlib, que es la librería que utilizaremos para hacer las visualizaciones

In [ ]:
import matplotlib.pyplot as plt

Para pintar el gráfico simplemente seleccionaremos la variable que queremos visualizar y utilizaremos la variable plot, pasándole el tamaño de la figura y el título que queremos ponerle a la gráfica.

La visualizamos haciendo un plt.show()

In [ ]:
df_comp['spx'].plot(figsize=(20,5), title = "S&P500 Prices")
plt.show() #
#plt.show() lo que hace es eliminar el mensaje que te añade previamente la serie

Podemos visualizar otra serie de tiempo

In [ ]:
df_comp['ftse'].plot(figsize=(20,5), title = "FTSE100 Prices")
plt.show()

También podemos añadir dos series temporales a la misma gráfica

In [ ]:
df_comp['spx'].plot(figsize=(20,5), title = "S&P500 Prices")
df_comp['ftse'].plot(figsize=(20,5), title = "FTSE100 Prices")
plt.title("S&P vs FTSE")
plt.show()

Ahora bien, como ves, en todas las gráficas que hemos pintado, el valor de la serie temporal en el eje x no es una fecha, sino que nos está poniendo el índice de cada uno de los valores.

Más adelante vamos a ver cómo cambiar este eje y añadir los valores de las fechas.

### QQ Plot

Otro de los gráficos que vamos a ver es el gráfico QQ Plot. Las QQ vienen de cuantil-cuantil.

El QQ plot lo que nos permite básicamente es explicar si un conjunto de datos se distribuye de cierta manera. Por defecto a menos que se indique otra cosa, la gráfica busca ajustarse a una distribución normal.

En este caso, la función que pinta este gráfico, está en el paquete scipy.
Vamos a importarlo:

In [ ]:
import scipy.stats

Para pintar la gráfica utilizaremos la función probplot

In [ ]:
scipy.stats.probplot(df_comp['spx'], plot =  plt)
#le pasamos la variable que queremos visualizar
#y el tipo de gráfico (en este caso de matplotlib)
plt.title("QQ Plot", size = 24) #tamaño título del gráfico
plt.show()

¿Cómo interpretamos este resultado?

La gráfica QQ plot lo que hace es tomar todos los valores de la variable, los ordena y en el eje Y indica los valores que toma. El eje X mide los cuantiles del conjunto de datos (es decir, a cuantas desviaciones de la media están los valores). La línea diagonal roja representa la forma que deberían seguir los datos si siguieran una distribución normal.

En este caso vemos que los datos no siguen una distribución normal (vemos que hay momentos en los que los valores de la serie se salen de la línea roja). Esto lo que generalmente suele pasar con las series de tiempo.

Ahora que ya tenemos toda esta información, vamos a ver cómo transformar los datos en una serie de tiempo.

### Transformar el texto en fechas

Como podemos comprobar, los valores de la variable date están en tipo caracter:

In [ ]:
type(df_comp['date'][0]) #selección del primer valor

Por tanto, debemos transformar esta variable en una de tipo fecha para poder luego utilizarla a la hora de pintar los ejes de las series de tiempo

Para hacer esta transformación vamos a utilizar la función to_datetime, de pandas, donde le indicaremos la variable que queremos modificar

In [ ]:
df_comp['date'] = pd.to_datetime(df_comp['date'], dayfirst = True)
#dayfirst = true lo que indica es que en el formato inicial de nuestros datos, el primer valor será el primero que sale
#07/01/1994

In [ ]:
type(df_comp['date'][0])

In [ ]:
df_comp.head()

Como ves, ahora la variable date ya han sido transformada en fechas.

Podemos también utilizar describe() para ver si todos los valores de las fechas son únicos, si no se repite ninguno, cual es el primer valor de la serie temporal, cual es el último...

In [ ]:
df_comp.describe()

### Colocar la variable date como índice de la serie temporal

Ahora vamos a situar la variable date como el índice de nuestro dataset, en lugar de tener el índice incremental que teníamos anteriormente:

In [ ]:
df_comp.head()

In [ ]:
df_comp.set_index("date", inplace=True)
df_comp.head()

### Configurar la frecuencia de la serie temporal

Como ves, la serie temporal no sigue la misma frecuencia. Del primer al segundo registro pasan tres dias, mientras que del segundo al tercero pasan únicamente uno.

Para trabajar con series temporales la frecuencia siempre tiene que ser la misma.

En python podemos ajustar las frecuencias de las series temporales utilizando la función asfreq().

- Si le pasamos una "h" como argumento serán datos con fecuencia horaria.
- Si le pasamos una "d", serán datos con frecuencia diaria.
- Si le pasamos datos con una "w" serán datos semanales
- Si le pasamos una "m" serán datos mensuales
- Si le pasamos una "a" serán datos anuales

Esto es básicamente porque durante los fines de semana no se registran datos en bolsa

[https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.asfreq.html](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.asfreq.html)

[https://www.datacamp.com/tutorial/pandas-resample-asfreq](https://www.datacamp.com/tutorial/pandas-resample-asfreq)

Vamos a ver primero los diarios:

In [ ]:
df_comp = df_comp.asfreq("d")
df_comp.head(10)

Como ves, nos crea nuevos registros donde en los días faltantes nos añade NaN.

Si queremos utilizar únicamente dias laborables, la función asfreq() tiene un argumento que es "b", que lo único que tiene en cuenta son los días laborables.

In [ ]:
df_comp = df_comp.asfreq("b")
df_comp.head(10)

### Trabajar con datos faltantes

In [ ]:
len(df_comp['spx'])

In [ ]:
df_comp['ftse'].plot(figsize=(20,5), title = "FTSE100 Prices")
plt.show()

Al crear los nuevos datos con las fechas laborables, **pueden haber registros que hayan incorporado datos faltante**s. Para comprobarlo, de nuevo podemos utilizar el isna() y sumar.

In [ ]:
df_comp.isna().sum()

Como ves, ahora si que tenemos valores faltantes.

Para solucionarlo, tenemos que encontrar la forma de rellenar esos valores.

Python incorpora las funciones ffill() y bfill() para conseguirlo, y permite rellenar de distintas formas.

Por ejemplo, si queremos rellenar con los valores posteriores de la serie temporal, podemos utilizar *ffill*:

In [ ]:
df_comp['spx'] = df_comp['spx'].fillna(method="ffill")
df_comp.isna().sum()

Si queremos rellenar con el valor anterior de la serie temporal, utilizamos *bfill*:

In [ ]:
#df_comp['ftse'] = df_comp['ftse'].fillna(method="bfill") # Deprecated
df_comp['ftse'] = df_comp['ftse'].bfill()
df_comp.isna().sum()

Podemos también rellenar valores faltantes con las medias:

In [ ]:
#df_comp['dax'] = df_comp['dax'].fillna(value=df_comp['dax'].mean())
#df_comp.isna().sum()

El problema de esto es que esta última forma de trabajar no es apropiada con series temporales, ya que sustituyendo por la media estamos perdiendo información de los patrones que sigue la serie temporal.

Únicamente tendría sentido si todos nuestros valores fluctuan alrededor de la media

In [ ]:
#df_comp['dax'] = df_comp['dax'].fillna(method = "bfill")
df_comp['dax'] = df_comp['dax'].bfill()
df_comp.isna().sum()

In [ ]:
#df_comp['nikkei'] = df_comp['nikkei'].fillna(method = "bfill")
df_comp['nikkei'] = df_comp['nikkei'].bfill()
df_comp.isna().sum()

### Simplificar el dataset

En este apartado vamos a ver cómo eliminar variables del dataset, que nos permita analizar cada una de las series temporales por separado

Lo primero que hacemos es guardarnos la variable del sp500 en una nueva variable y posteriormente eliminar el resto

In [ ]:
df_comp["market_value"] = df_comp['spx']

In [ ]:
del df_comp["spx"]
del df_comp["dax"]
del df_comp["ftse"]
del df_comp["nikkei"]

In [ ]:
df_comp.head()

### Dividir los datos en train y test

Como ya sabemos, una serie temporal **debe mantener el orden de los valores**. Para separar el train del test vamos a hacerlo de la siguiente forma:

1) Calculamos la longitud de la serie temporal con un 80% de los datos, que serán los datos de entrenamiento

In [ ]:
size = int(len(df_comp)*0.8)
size

2) Seleccionamos los datos hasta ese valor para el conjunto de train (recordamos que en python es ese valor, menos uno)

In [ ]:
df = df_comp.iloc[:size]
len(df)

3) Seleccionamos los datos desde ese valor para el conjunto de test

In [ ]:
df_test = df_comp.iloc[size:]
len(df_test)

##Características de una serie temporal

### Ruido blanco

Ahora vamos a generar ruido blanco, y vamos a almacenar sus valores en una variable que se llama rb.

Para crear ruido blanco vamos a utilizar la función normal, dentro de la librería numpy, que genera X valores aleatorios siguiendo una distribución normal, donde le indicamos como argumento la media y la desviación estandar de dicha distribución.

In [ ]:
rb = np.random.normal(loc = df['market_value'].mean(),
                      scale = df['market_value'].std(),
                      size = len(df))
#Size -> Número de datos a crear -> tantos como longitud del dataset
#scale -> desviación estandar de la distribución
#loc -> media de la disttibución

Una vez creados estos valores, los almacenamos en una variable en el conjunto de datos de entrenamiento:

In [ ]:
df['rb'] = rb

Por tanto, ahora tenemos dos variables en el conjunto de datos. Podemos ver sus estadísticos con describe:

In [ ]:
df.describe()

Vamos a pintar ahora la serie temporal del ruido blanco:

In [ ]:
df.rb.plot(figsize = (20,5))
plt.title("Ruido blanco serie temporal", size= 24)
plt.show()

Como ves, el ruido blanco tiene un comportamiento aleatorio alrededor del valor medio y con una varianza constante.

Y por otro lado vamos a pintar la serie temporal sin el ruido:

In [ ]:
df['market_value'].plot(figsize=(20,5))
plt.title("S&P Precios", size = 24)
plt.ylim(0,2300)
plt.show()

Este ruido blanco, ahora lo hemos creado de forma sintética a partir de la serie temporal de sp500 para ver qué pinta tienen: una media constante, una varianza constante

### Estacionariedad

Para analizar una serie temporal (y aplicar los modelos que veremos a continuación), es importante definir si los datos siguen un proceso estacionario (es decir, media constante, varianza constante).

Para comprobar si una serie temporal es estacionario o no utilizamos la prueba Dickey-Fuller.

Como ya hemos visto, la prueba está basada en un contraste de hipótesis, donde la hipótesis nula va a ser que la serie no es estacionaria y la hipotesis alternativa, que si es estacionaria.

En Python el test de Dickey-Fuller se puede hacer con la función adfuller, que está en el módulo de estadística statmodel.

Lo que hacemos es pasarle los datos de nuestra serie temporal en train a la función:

In [ ]:
sts.adfuller(df['market_value'])

-1.736984745235244 -> Valor del estadístico de contraste que nos devuelve el test.

0.41216456967706194 -> p-valor

'1%': -3.431658008603046

'10%': -2.567077669247375

'5%': -2.862117998412982

Son los valores de cada uno de los intervalos de confianza a elegir. Si el valor del estadístico es menor que el valor crítico, rechazamos la hipótesis nula.
Si rechazamos la hipótesis nula es que la serie es estacionaria.

En este caso la serie **no** es estacionaria para ninguno de los casos.

Podemos repetirlo en la variable de la serie temporal que hemos creado anteriormente de ruido blanco, que por definición anteriormente hemos visto que debería ser una serie estacionaria

In [ ]:
sts.adfuller(df.rb)

El valor del estadístico es menor que cualquier valor crítico y el p-valor es menor del nivel de confianza habitual, 0.05, por lo que en este caso se rechaza la H0 (no estacionaria) y se considera que la serie es estacionaria (lo cual ya sabíamos;)

### Estacionalidad

Para estudiar la estacionalidad habíamos hablado que el método clásico que solía utilizarse era la dsecomposición de la serie temporal en: tendencia, estacionalidad y residuos).

Lo primero que tenemos que definir es si la serie es aditiva o multiplicativa.

El calculo de componentes estacionales en python lo podemos hacer con la función `seasonal_decompose()` de la librería `statsmodels`



In [ ]:
#aplicamos primero aditiva y almacenamos los datos en una variable:
s_dec_additive = seasonal_decompose(df['market_value'], model = "additive")

#visualizamos los datos de la variable:
s_dec_additive.plot()
plt.show()

Lo que nos devuelve esta función es, en primer lugar la serie temporal, en segundo lugar la tendencia, en tercer lugar su componente estacional y, por último, los residuos (que tienen un aspecto muy cercano a ruido blanco)

In [ ]:
s_dec_multiplicative = seasonal_decompose(df['market_value'], model = "multiplicative")
s_dec_multiplicative.plot()
plt.show()

### Autocorrelación (ACF)

Para calcular la autocorrelación en la serie temporal del sp500, vamos a utilizar la función `plot_acf()`, que permite dibujar la función de autocorrelación.

Al método le tenemos que pasar la variable que nos interesa y los lags (los retrasos, los puntos anteriores que va a tener en cuenta la función para calcular la autocorrelación). Por defecto si no ponemos nada el modelo tiene en cuenta toda la serie. Por lo general suelen utilizarse 40 retrasos.

Y después, el argumento zero lo que hace es si tenemos en cuenta el valor del periodo cero (es decir, el primer valor consigo mismo). Eso no nos interesa.

In [ ]:
sgt.plot_acf(df['market_value'], lags = 40, zero = False)
plt.title("ACF S&P", size = 24)
plt.show()

Los valores pueden ir desde -1 hasta 1, 1 es la máxima correlación y -1 es la máxima correlación negativa.

En esta gráfica vemos que existe una alta correlación entre el primer valor y los 39 siguientes. La primera línea indica la correlación del valor actual con uno atrás en el tiempo, la segunda línea indica la correlación del valor actual con dos valores atrás en el tiempo.

La región azul indica que el valor de cada línea es significativa si estamos por encima.

Podemos repetir el mismo proceso con el ruido blanco. Como ves, aquí ningún valor está correlacionado. Cada uno está en una posición y además todos están por debajo de la franja de autocorrelación.

In [ ]:
sgt.plot_acf(df.rb, lags = 40, zero = False)
plt.title("ACF Ruido blanco", size = 24)
plt.show()

Como es de esperar, no hay ninguna autocorrelación entre los valores del ruido blanco.

### Autocorrelación Parcial (PACF)

Vamos a ver ahora la autocorrelación parcial, que recordemos que mide la similitud

In [ ]:
sgt.plot_pacf(df['market_value'], lags = 40,
              zero = False,
              method = ('ols'))
#ols: mínimos cuadrados ordinarios

plt.title("PACF S&P", size = 24)
plt.show()

Lo que observamos en este gráfico, lo primero,en azul, es el área de significación. En este caso únicamente los primeros valores son significativos. También vemos valores correlacionados negativos.

In [ ]:
sgt.plot_pacf(df.rb, lags = 40,
              zero = False,
              method = ('ols'))
plt.title("PACF Ruido blanco", size = 24)
plt.show()

##**Resumen**

Para trabajar adecuadamente con datos de series temporales hemos de realizar una serie de operaciones de preprocesado. Algunas operaciones son comunes a cualquier análisis de datos y otras son más específicas por la naturaleza particular de estos datos:

+ Explorar los datos: campos, registros, dimensiones, tipos de datos, calidad.
+ Visualizarlos datos y otros gráficos útiles
+ Descomposición: estacionariedad, tendencia, estacionalidad, etc.
+ Análisis de ACF y PACF

Llegado este punto tenemos una mejor idea de qué datos tenemos y cómo podemos abordar siguientes fases de análisis y/o predicción.

#EOF (End Of File)